In [ ]:
#! Python3
# -*- coding: utf-8 -*-
# 異分野共著 process III : idp check

import csv
import pandas  as pd
import numpy as np

#1. dataファイル, author_asjc27Listを開く
kudf = pd.read_csv('Publications-KU2017.csv')
asjcdf = pd.read_csv('AuthorIDwithasjc27List.csv', dtype = 'object')

#duplication delate
asjcdf['document count'] = asjcdf['document count'].astype(int)
asjcdf['asjc27'] = asjcdf['asjc27'].astype(int)
print('Authors with less than 5 papers: ' + str(len(asjcdf[asjcdf['document count'] < 5])))
asjcdf = asjcdf.drop_duplicates()
print('Authors with asjc27 code: ' + str(len(asjcdf[asjcdf['asjc27'] != 0])))

#2. 異分野共著かどうか判別
#2-1. Author Idsを分野に置き換える
papers = []
for i, row in kudf.iterrows():
    bunya = []
    for j, aid in asjcdf.iterrows():
        if aid['Author ID'] in row['Scopus Author Ids']:
            bunya.append(aid['asjc27'])
        else:
            continue

    #2-2. 著者分野の重複削除
    asjc27 = set(bunya)

    #2-3. 異分野共著かどうかを判別（異分野共著ならXを入れる）
    if len(asjc27) < 2:
        idp = ''
    elif len(asjc27) == 2:
        if 0 in asjc27:
            idp = ''
        else:
            idp = 'X'
    else:
        idp = 'X'

    print(str(i) + ', ' + str(row['EID']) + ': ' + str(idp))
    paper = [row['EID'], asjc27, len(asjc27), idp]
    papers.append(paper)

#idpList
#本学全体値
papersdf = pd.DataFrame(papers, columns=['EID', 'asjc27', 'Number of asjc', 'idp'])
pubdf = pd.merge(papersdf, kudf, how = 'left', on = 'EID')
papersdf.to_csv('Publications_KU2017_idp.csv', index=False)
print('Interdeciplinary : ' + str(papersdf[papersdf['idp'] == 'X'].count()) + ' in all KU publications.')
